In [1]:
import numpy as np
import pandas as pd
# from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
# import sklearn

In [2]:
import os

---

In [3]:
train  = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')

train = train.drop(['index'],axis = 1)
test =test.drop(['index'],axis = 1)

In [4]:
concat_features = train

mask = concat_features.introelapse > (concat_features.introelapse.quantile(0.75) + 1.5 * (concat_features.introelapse.quantile(0.75) - concat_features.introelapse.quantile(0.25)))
concat_features.introelapse[mask] = concat_features.introelapse.quantile(0.75)

mask = concat_features.testelapse > (concat_features.testelapse.quantile(0.75) + 1.5 * (concat_features.testelapse.quantile(0.75) - concat_features.testelapse.quantile(0.25)))
concat_features.testelapse[mask] = concat_features.testelapse.quantile(0.75)

mask = concat_features.surveyelapse > (concat_features.surveyelapse.quantile(0.75) + 1.5 * (concat_features.surveyelapse.quantile(0.75) - concat_features.surveyelapse.quantile(0.25)))
concat_features.surveyelapse[mask] = concat_features.surveyelapse.quantile(0.75)

train = concat_features



concat_features = test

mask = concat_features.introelapse > (concat_features.introelapse.quantile(0.75) + 1.5 * (concat_features.introelapse.quantile(0.75) - concat_features.introelapse.quantile(0.25)))
concat_features.introelapse[mask] = concat_features.introelapse.quantile(0.75)

mask = concat_features.testelapse > (concat_features.testelapse.quantile(0.75) + 1.5 * (concat_features.testelapse.quantile(0.75) - concat_features.testelapse.quantile(0.25)))
concat_features.testelapse[mask] = concat_features.testelapse.quantile(0.75)

mask = concat_features.surveyelapse > (concat_features.surveyelapse.quantile(0.75) + 1.5 * (concat_features.surveyelapse.quantile(0.75) - concat_features.surveyelapse.quantile(0.25)))
concat_features.surveyelapse[mask] = concat_features.surveyelapse.quantile(0.75)

test = concat_features


In [5]:
train.loc[train['age'] == 0, 'age'] = train['age'].mean()
test.loc[test['age'] == 0, 'age'] = train['age'].mean()

In [6]:
train.loc[train['age'] > 65, 'age'] = 0
test.loc[test['age'] > 65, 'age'] = 0

In [7]:
train.loc[train['familysize'] > 6, 'familysize'] = train['familysize'].mode()[0]
test.loc[test['familysize'] > 6, 'familysize'] = train['familysize'].mode()[0]

In [8]:
for name in ['education', 'urban', 'gender', 'engnat', 'hand', 'religion',
            'voted', 'married']:

    mergedata = pd.get_dummies(train[name], prefix = name)
    train = pd.concat([train, mergedata], axis=1)
    train = train.drop([name], axis = 1)

In [9]:
for name in ['education', 'urban', 'gender', 'engnat', 'hand', 'religion',
            'voted', 'married']:

    mergedata = pd.get_dummies(test[name], prefix = name)
    test = pd.concat([test, mergedata], axis=1)
    test = test.drop([name], axis = 1)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 94 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Q1             14959 non-null  float64
 1   Q2             14931 non-null  float64
 2   Q3             14950 non-null  float64
 3   Q4             14929 non-null  float64
 4   Q5             14962 non-null  float64
 5   Q6             14952 non-null  float64
 6   Q7             14924 non-null  float64
 7   Q8             14952 non-null  float64
 8   Q9             14944 non-null  float64
 9   Q10            14928 non-null  float64
 10  Q11            14941 non-null  float64
 11  Q12            14933 non-null  float64
 12  Q13            14960 non-null  float64
 13  Q14            14964 non-null  float64
 14  Q15            14955 non-null  float64
 15  Q16            14967 non-null  float64
 16  Q17            14963 non-null  float64
 17  Q18            14937 non-null  float64
 18  Q19   

In [11]:
from sklearn.preprocessing import LabelEncoder
ex = LabelEncoder()

con8 = train.country.value_counts().keys()[:6]

for i in range(train.count().max()):
    if(train.country.values[i] not in con8):
        train.country.values[i] = "X"

for i in range(test.count().max()):
    if(test.country.values[i] not in con8):
        test.country.values[i] = "X"

train.country = ex.fit_transform(train.country)

test.country = ex.fit_transform(test.country)

---

In [12]:
# pip install pycaret

In [13]:
from pycaret.classification import *

model = setup(data = train, target = 'nerdiness',use_gpu=True,session_id=2023, normalize = True, normalize_method = 'zscore',transformation=True,train_size=0.85)

,Description,Value
0,session_id,2023
1,Target,nerdiness
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(15000, 94)"
5,Missing Values,True
6,Numeric Features,38
7,Categorical Features,55
8,Ordinal Features,False
9,High Cardinality Features,False


In [14]:
rf = create_model('rf', cross_validation = False)
et = create_model('et', cross_validation = False)
lightgbm = create_model('lightgbm', cross_validation = False)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7392,0.823,0.8101,0.7318,0.769,0.4714,0.4746


In [15]:
blended = blend_models(estimator_list = [rf,lightgbm], fold = 10, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7796,0.8550,0.8324,0.7849,0.8079,0.5499,0.5513
1,0.7914,0.8702,0.8366,0.7984,0.8171,0.5747,0.5755
2,0.8008,0.8812,0.8549,0.8008,0.8270,0.5928,0.5946
3,0.7859,0.8656,0.8507,0.7834,0.8157,0.5613,0.5640
4,0.7671,0.8537,0.8507,0.7597,0.8027,0.5206,0.5256
5,0.7592,0.8455,0.8254,0.7620,0.7924,0.5069,0.5092
6,0.7757,0.8552,0.8465,0.7725,0.8078,0.5398,0.5431
7,0.7780,0.8509,0.8463,0.7752,0.8092,0.5452,0.5482
8,0.7820,0.8488,0.8477,0.7795,0.8122,0.5534,0.5562


In [16]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7859,0.8625,0.8557,0.7701,0.8107,0.5658,0.5700


In [17]:
final_model = finalize_model(blended)

In [18]:
predictions = predict_model(final_model, data = test)

In [19]:
predictions['Score']

0        0.5814
1        0.7744
2        0.8213
3        0.5629
4        0.7289
          ...  
35447    0.7193
35448    0.8221
35449    0.8682
35450    0.6459
35451    0.7527
Name: Score, Length: 35452, dtype: float64

In [20]:
predictions['Label']

0        0
1        1
2        1
3        1
4        1
        ..
35447    1
35448    1
35449    1
35450    0
35451    1
Name: Label, Length: 35452, dtype: int32

In [21]:
predictions.loc[predictions['Label'] == 0, 'Score'] = 1 - predictions.loc[predictions['Label'] == 0, 'Score']

In [22]:
predictions['Score']

0        0.4186
1        0.7744
2        0.8213
3        0.5629
4        0.7289
          ...  
35447    0.7193
35448    0.8221
35449    0.8682
35450    0.3541
35451    0.7527
Name: Score, Length: 35452, dtype: float64

In [23]:
sub = predictions['Score']

In [24]:
sub

0        0.4186
1        0.7744
2        0.8213
3        0.5629
4        0.7289
          ...  
35447    0.7193
35448    0.8221
35449    0.8682
35450    0.3541
35451    0.7527
Name: Score, Length: 35452, dtype: float64

In [25]:
submission = pd.read_csv('./data/sample_submission.csv')

submission['nerdiness'] = sub

In [26]:
submission

,index,nerdiness
0,0,0.4186
1,1,0.7744
2,2,0.8213
3,3,0.5629
4,4,0.7289
...,...,...
35447,35447,0.7193
35448,35448,0.8221
35449,35449,0.8682
35450,35450,0.3541


In [27]:
submission.to_csv("./submit/submit13.csv", index = False)